In [3]:
import csv
import random

# 读取CSV文件
input_file = 'Y_train.csv'
with open(input_file, 'r') as f:
    reader = csv.reader(f)
    data = list(reader)

# 提取标题
title = data[0]

# 提取除标题外的数据
rows = data[1:]

# 创建24个新的CSV文件
for i in range(24):
    # 提取符合条件的行
    extracted_rows = [rows[j] for j in range(len(rows)) if (j - i) % 24 == 0]
    
    # 将提取的行写入新的CSV文件
    output_file = f'./generated_data/output_{i+1}.csv'
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(title)
        writer.writerows(extracted_rows)

    # 对列中的每个数进行-10%至+10%的随机变化，共变化五次
    with open(output_file, 'r') as f:
        reader = csv.reader(f)
        data = list(reader)
    
    title_row = data[0]
    rows_data = data[1:]
    
    for row in rows_data:
        original_value = float(row[0])
        for j in range(1, 2):
            random_factor = random.uniform(-0.15, 0.15)
            new_value = original_value * (1 + random_factor)
            row.append(f"{new_value:.2f}")
    
    # 将变化后的数据写入CSV文件
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(title_row + ['Change1'])
        writer.writerows(rows_data)

print("处理完成，已生成24个新的CSV文件。")

处理完成，已生成24个新的CSV文件。


In [4]:
import csv
import numpy as np
import os

# mpc_bus数据
mpc_bus = [
    [1, 51], [2, 20], [3, 39], [4, 39], [5, 0], [6, 52], [7, 19], [8, 28], [9, 0], [10, 0],
    [11, 70], [12, 47], [13, 34], [14, 14], [15, 90], [16, 25], [17, 11], [18, 60], [19, 45], [20, 18],
    [21, 14], [22, 10], [23, 7], [24, 13], [25, 0], [26, 0], [27, 71], [28, 17], [29, 24], [30, 0],
    [31, 43], [32, 59], [33, 23], [34, 59], [35, 33], [36, 31], [37, 0], [38, 0], [39, 27], [40, 66],
    [41, 37], [42, 96], [43, 18], [44, 16], [45, 53], [46, 28], [47, 34], [48, 20], [49, 87], [50, 17],
    [51, 17], [52, 18], [53, 23], [54, 113], [55, 63], [56, 84], [57, 12], [58, 12], [59, 277], [60, 78],
    [61, 0], [62, 77], [63, 0], [64, 0], [65, 0], [66, 39], [67, 28], [68, 0], [69, 0], [70, 66],
    [71, 0], [72, 12], [73, 6], [74, 68], [75, 47], [76, 68], [77, 61], [78, 71], [79, 39], [80, 130],
    [81, 0], [82, 54], [83, 20], [84, 11], [85, 24], [86, 21], [87, 0], [88, 48], [89, 0], [90, 163],
    [91, 10], [92, 65], [93, 12], [94, 30], [95, 42], [96, 38], [97, 15], [98, 34], [99, 42], [100, 37],
    [101, 22], [102, 5], [103, 23], [104, 38], [105, 31], [106, 43], [107, 50], [108, 2], [109, 8], [110, 39],
    [111, 0], [112, 68], [113, 6], [114, 8], [115, 22], [116, 184], [117, 20], [118, 33]
]

# 计算负荷比例
ratio_bus = np.zeros(118)
for i in range(118):
    ratio_bus[i] = mpc_bus[i][1] / sum(mpc_bus[j][1] for j in range(118))

# 创建父文件夹 './hour_1'，如果不存在
parent_folder = './hour'
if not os.path.exists(parent_folder):
    os.makedirs(parent_folder)

# 处理 output_1.csv 到 output_24.csv
for file_num in range(1, 25):
    # 读取 output_{file_num}.csv 文件
    input_file = f'./generated_data/output_{file_num}.csv'
    with open(input_file, 'r') as f:
        reader = csv.reader(f)
        data = list(reader)

    # 提取标题和数据
    title = data[0]
    rows = data[1:]

    # 创建子文件夹 './hour_1/{file_num}'，如果不存在
    folder_path = f'{parent_folder}/{file_num}'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # 分配第一列数据
    allocated_data = []
    for row in rows:
        total_value = float(row[0])  # 第一列数据
        allocated_row = [total_value * ratio_bus[i] for i in range(118)]
        allocated_data.append(allocated_row)

    # 保存分配后的第一列数据
    output_file = f'{folder_path}/allocated_output_perfect.csv'
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Node_' + str(i+1) for i in range(118)])  # 写入节点名标题
        writer.writerows(allocated_data)
    print(f"数据分配完成，已保存到{output_file}文件中。")

    # 分配第二至第六列数据并保存
    for col in range(1, 2):
        allocated_data = []
        for row in rows:
            total_value = float(row[col])  # 读取第col列数据
            allocated_row = [total_value * ratio_bus[i] for i in range(118)]
            allocated_data.append(allocated_row)

        # 保存分配后的数据
        output_file = f'{folder_path}/allocated_output_change{col}.csv'
        with open(output_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Node_' + str(i+1) for i in range(118)])  # 写入节点名标题
            writer.writerows(allocated_data)

        print(f"数据分配完成，已保存到{output_file}文件中。")


数据分配完成，已保存到./hour/1/allocated_output_perfect.csv文件中。
数据分配完成，已保存到./hour/1/allocated_output_change1.csv文件中。
数据分配完成，已保存到./hour/2/allocated_output_perfect.csv文件中。
数据分配完成，已保存到./hour/2/allocated_output_change1.csv文件中。
数据分配完成，已保存到./hour/3/allocated_output_perfect.csv文件中。
数据分配完成，已保存到./hour/3/allocated_output_change1.csv文件中。
数据分配完成，已保存到./hour/4/allocated_output_perfect.csv文件中。
数据分配完成，已保存到./hour/4/allocated_output_change1.csv文件中。
数据分配完成，已保存到./hour/5/allocated_output_perfect.csv文件中。
数据分配完成，已保存到./hour/5/allocated_output_change1.csv文件中。
数据分配完成，已保存到./hour/6/allocated_output_perfect.csv文件中。
数据分配完成，已保存到./hour/6/allocated_output_change1.csv文件中。
数据分配完成，已保存到./hour/7/allocated_output_perfect.csv文件中。
数据分配完成，已保存到./hour/7/allocated_output_change1.csv文件中。
数据分配完成，已保存到./hour/8/allocated_output_perfect.csv文件中。
数据分配完成，已保存到./hour/8/allocated_output_change1.csv文件中。
数据分配完成，已保存到./hour/9/allocated_output_perfect.csv文件中。
数据分配完成，已保存到./hour/9/allocated_output_change1.csv文件中。
数据分配完成，已保存到./hour/10/allocated_output_perfect.

In [ ]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import os
import csv
from concurrent.futures import ThreadPoolExecutor, as_completed
# 优化模型函数
def calculate_optimization_cost_two_stage(P_l_predict, Y_real, topo, unit, reserve_up=150, reserve_down=150, c=9000, c_s=1750):
    """
    两阶段优化模型函数，输入数据为单列，按照 ratio_bus 分配到 118 个节点。
    
    Args:
        P_l_predict (np.array): 负荷预测值，形状为 (样本数,)
        Y_real (np.array): 实际负荷值，形状为 (样本数,)
        topo (pd.DataFrame): 拓扑数据
        unit (pd.DataFrame): 机组参数数据
        reserve_up (int): 上调备用要求
        reserve_down (int): 下调备用要求
        c (int): 负荷削减成本系数
        c_s (int): 松弛变量成本系数
        
    Returns:
        float: 总优化成本
    """
    P_l_predict = np.array(P_l_predict).T    
    Y_real = np.array(Y_real).T
    
    nodes = topo.shape[0]
    
    # ========== 日前调度优化问题 ==========
    model_DA = gp.Model()
    
    # 定义决策变量
    P_c = model_DA.addVars(nodes,  lb=0, name='P_c')       # 日前机组出力
    R_U = model_DA.addVars(nodes,  lb=0, name='R_U')       # 上调备用
    R_D = model_DA.addVars(nodes,  lb=0, name='R_D')       # 下调备用
    delta_DA = model_DA.addVars(nodes,  lb=-180, ub=180, name='delta_DA') # 日前相角
    
    # 目标函数
    obj_DA = gp.quicksum(
        R_U[i]*unit.iloc[i,5] + 
        R_D[i]*unit.iloc[i,6] + 
        P_c[i]*unit.iloc[i,2] 
        for i in range(nodes) 
    )
    model_DA.setObjective(obj_DA, GRB.MINIMIZE)
    
    # 约束条件
        # 备用需求约束
    model_DA.addConstr(gp.quicksum(R_U[i] for i in range(nodes)) == reserve_up)
    model_DA.addConstr(gp.quicksum(R_D[i] for i in range(nodes)) == reserve_down)
        # 机组备用容量约束
    for i in range(nodes):
        model_DA.addConstr(R_U[i] <= unit.iloc[i,3])  # 上调备用限制
        model_DA.addConstr(R_D[i] <= unit.iloc[i,4])  # 下调备用限制
        # 日前市场功率平衡
    for t in range(nodes):
        model_DA.addConstr(
            P_c[t] - P_l_predict[t] - 
            gp.quicksum(topo.iloc[t,j]*(delta_DA[t]-delta_DA[j]) for j in range(nodes)) == 0
        )
    model_DA.addConstr(delta_DA[0] == 0)  # 参考节点相角
        # 日前机组出力约束
    for i in range(nodes):
        model_DA.addConstr(P_c[i] <= unit.iloc[i,1] - R_U[i])  # 最大出力
        model_DA.addConstr(P_c[i] >= R_D[i])                   # 最小出力
        # 日前线路容量约束
    for i in range(nodes):
        for j in range(nodes):
            if i != j and topo.iloc[i,j] < -0.01:
                model_DA.addConstr(topo.iloc[i,j] * (delta_DA[i] - delta_DA[j]) <= 175)
                model_DA.addConstr(topo.iloc[i,j] * (delta_DA[i] - delta_DA[j]) >= -175)
    
    model_DA.optimize()
    
    # ========== 实时调度优化问题 ==========
    model_RT = gp.Model()
    
    # 定义决策变量
    r_U = model_RT.addVars(nodes,  lb=0, name='r_U')       # 实际上调调整
    r_D = model_RT.addVars(nodes,  lb=0, name='r_D')       # 实际下调调整
    P_lsh = model_RT.addVars(nodes,  lb=0, name='P_lsh')   # 负荷削减
    delta_RT = model_RT.addVars(nodes,  lb=-180, ub=180, name='delta_RT') # 实时相角
    s = model_RT.addVars(nodes,  lb=0, name='s')           # 松弛变量
    
    # 目标函数
    obj_RT = gp.quicksum(
        (r_U[i]-r_D[i])*unit.iloc[i,2] 
        for i in range(nodes) 
    )
    obj_RT += gp.quicksum(P_lsh[i]*c for i in range(nodes) )
    obj_RT += gp.quicksum(s[i]*c_s for i in range(nodes) )
    model_RT.setObjective(obj_RT, GRB.MINIMIZE)
    
    # 约束条件
        # 实时市场功率平衡
    for t in range(nodes):
        model_RT.addConstr(
            r_U[t] - r_D[t] - s[t] - Y_real[t] + P_l_predict[t] + P_lsh[t] - 
            gp.quicksum(
                topo.iloc[t,j]*(delta_RT[t]-delta_RT[j]) - 
                topo.iloc[t,j]*(delta_DA[t].X-delta_DA[j].X) 
                for j in range(nodes)
            ) == 0
        )
        # 实时备用调整约束
    for i in range(nodes):
        model_RT.addConstr(r_U[i] <= R_U[i].X)  # 实际上调调整不超过备用
        model_RT.addConstr(r_D[i] <= R_D[i].X)  # 实际下调调整不超过备用
    model_RT.addConstr(delta_RT[0] == 0)  # 参考节点相角
        # 实时线路容量约束
    for i in range(nodes):
        for j in range(nodes):
            if i != j and topo.iloc[i,j] < -0.01:
                model_RT.addConstr(topo.iloc[i,j] * (delta_RT[i] - delta_RT[j]) <= 175)
                model_RT.addConstr(topo.iloc[i,j] * (delta_RT[i] - delta_RT[j]) >= -175)
    
    model_RT.optimize()
    
    # 计算总成本
    total_cost = model_DA.ObjVal + model_RT.ObjVal
    
    # 打印两阶段成本值和总成本值
    print(f"日前调度成本: {model_DA.ObjVal}")
    print(f"实时调度成本: {model_RT.ObjVal}")
    print(f"总成本: {total_cost}")
    
    # 释放模型资源
    model_DA.dispose()
    model_RT.dispose()
    
    return total_cost
# System parameters (topology data and generator unit data)
topo = pd.read_excel('118nodes_system.xlsx', sheet_name='topology', index_col=None, header=None) 
unit = pd.read_excel('118nodes_system.xlsx', sheet_name='unit')
# 遍历 hour 文件夹中的 1-24 文件夹
hour_folder = './hour'
for hour in range(6, 13):
    hour_dir = os.path.join(hour_folder, str(hour))
    if not os.path.exists(hour_dir):
        continue
    
    # 读取 allocated_output_perfect 文件
    perfect_file = os.path.join(hour_dir, 'allocated_output_perfect.csv')
    if not os.path.exists(perfect_file):
        continue
    
    # 读取 allocated_output_perfect 文件的内容
    with open(perfect_file, 'r') as f_perfect:
        reader_perfect = csv.reader(f_perfect)
        next(reader_perfect)  # 跳过标题行
        perfect_data = [list(map(float, row)) for row in reader_perfect]
    
    # 使用 ThreadPoolExecutor 并行计算无偏预测的总成本
    perfect_costs = []
    with ThreadPoolExecutor() as executor:
        # 使用 map 确保顺序一致
        perfect_costs = list(executor.map(lambda x: calculate_optimization_cost_two_stage(x, x, topo, unit), perfect_data))
    
    # 保存无偏预测的总成本到 CSV 文件
    perfect_cost_file = os.path.join(hour_dir, 'perfect_costs.csv')
    with open(perfect_cost_file, 'w', newline='') as f_perfect_cost:
        writer = csv.writer(f_perfect_cost)
        writer.writerow(['Perfect_Cost'])
        writer.writerows([[cost] for cost in perfect_costs])
    print(f"无偏预测的总成本已保存到 {perfect_cost_file} 文件中。")
    
    # 读取 allocated_output_change1-5 的 CSV 文件
    cost_differences = []  # 存储五个文件的成本差值
    for file_prefix in ['allocated_output_change1']:
        input_file = os.path.join(hour_dir, f'{file_prefix}.csv')
        if not os.path.exists(input_file):
            continue
        
        # 读取 CSV 文件
        with open(input_file, 'r') as f_in:
            reader = csv.reader(f_in)
            next(reader)  # 跳过标题行
            data = [list(map(float, row)) for row in reader]
        
        # 检查数据行数是否一致
        if len(data) != len(perfect_data):
            print(f"警告：{input_file} 文件的行数与 allocated_output_perfect.csv 不一致，跳过该文件。")
            continue
        
        # 使用 ThreadPoolExecutor 并行计算总成本
        total_costs = []
        with ThreadPoolExecutor() as executor:
            # 使用 map 确保顺序一致
            total_costs = list(executor.map(lambda x: calculate_optimization_cost_two_stage(x[0], x[1], topo, unit), zip(data, perfect_data)))
        
        # 计算成本差值
        cost_diff = [total_costs[i] - perfect_costs[i] for i in range(len(total_costs))]
        cost_differences.append(cost_diff)
    
    # 将五个文件的成本差值合并为一个 CSV 文件
    if cost_differences:
        output_file = os.path.join(hour_dir, 'cost_differences.csv')
        with open(output_file, 'w', newline='') as f_out:
            writer = csv.writer(f_out)
            writer.writerow(['Change1', 'Change2', 'Change3', 'Change4', 'Change5'])
            writer.writerows(zip(*cost_differences))
        print(f"成本差值已保存到 {output_file} 文件中。")
    
    # 输出第 hour 文件夹已处理完成
    print(f"第 {hour} 文件夹已处理完成。")

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 10 physical cores, 16 logical processors, using up to 16 threads
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Optimize a model with 1309 rows, 472 columns and 2971 nonzeros
Thread count: 10 physical cores, 16 logical processors, using up to 16 threads
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Optimize a model with 1309 rows, 472 columns and 2971 nonzeros
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Model fingerprint: 0x9f648469
Thread count: 10 physical cores, 16 logical processors, using up to 16 threads
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 10 physical cores, 16 logical processors, using up to 16 threads
Thread count: 10 physical cores, 16 logical processors, using up to 16 threads
Thread co

In [2]:
#顺序执行
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import os
import csv

# 优化模型函数
def calculate_optimization_cost_two_stage(P_l_predict, Y_real, topo, unit, reserve_up=150, reserve_down=150, c=9000, c_s=1750):
    """
    两阶段优化模型函数，输入数据为单列，按照 ratio_bus 分配到 118 个节点。
    
    Args:
        P_l_predict (np.array): 负荷预测值，形状为 (样本数,)
        Y_real (np.array): 实际负荷值，形状为 (样本数,)
        topo (pd.DataFrame): 拓扑数据
        unit (pd.DataFrame): 机组参数数据
        reserve_up (int): 上调备用要求
        reserve_down (int): 下调备用要求
        c (int): 负荷削减成本系数
        c_s (int): 松弛变量成本系数
        
    Returns:
        float: 总优化成本
    """
    P_l_predict = np.array(P_l_predict).T    
    Y_real = np.array(Y_real).T
    
    nodes = topo.shape[0]
    
    # ========== 日前调度优化问题 ==========
    model_DA = gp.Model()
    
    # 定义决策变量
    P_c = model_DA.addVars(nodes,  lb=0, name='P_c')       # 日前机组出力
    R_U = model_DA.addVars(nodes,  lb=0, name='R_U')       # 上调备用
    R_D = model_DA.addVars(nodes,  lb=0, name='R_D')       # 下调备用
    delta_DA = model_DA.addVars(nodes,  lb=-180, ub=180, name='delta_DA') # 日前相角
    
    # 目标函数
    obj_DA = gp.quicksum(
        R_U[i]*unit.iloc[i,5] + 
        R_D[i]*unit.iloc[i,6] + 
        P_c[i]*unit.iloc[i,2] 
        for i in range(nodes) 
    )
    model_DA.setObjective(obj_DA, GRB.MINIMIZE)
    
    # 约束条件
        # 备用需求约束
    model_DA.addConstr(gp.quicksum(R_U[i] for i in range(nodes)) == reserve_up)
    model_DA.addConstr(gp.quicksum(R_D[i] for i in range(nodes)) == reserve_down)
        # 机组备用容量约束
    for i in range(nodes):
        model_DA.addConstr(R_U[i] <= unit.iloc[i,3])  # 上调备用限制
        model_DA.addConstr(R_D[i] <= unit.iloc[i,4])  # 下调备用限制
        # 日前市场功率平衡
    for t in range(nodes):
        model_DA.addConstr(
            P_c[t] - P_l_predict[t] - 
            gp.quicksum(topo.iloc[t,j]*(delta_DA[t]-delta_DA[j]) for j in range(nodes)) == 0
        )
    model_DA.addConstr(delta_DA[0] == 0)  # 参考节点相角
        # 日前机组出力约束
    for i in range(nodes):
        model_DA.addConstr(P_c[i] <= unit.iloc[i,1] - R_U[i])  # 最大出力
        model_DA.addConstr(P_c[i] >= R_D[i])                   # 最小出力
        # 日前线路容量约束
    for i in range(nodes):
        for j in range(nodes):
            if i != j and topo.iloc[i,j] < -0.01:
                model_DA.addConstr(topo.iloc[i,j] * (delta_DA[i] - delta_DA[j]) <= 175)
                model_DA.addConstr(topo.iloc[i,j] * (delta_DA[i] - delta_DA[j]) >= -175)
    
    model_DA.optimize()
    
    # ========== 实时调度优化问题 ==========
    model_RT = gp.Model()
    
    # 定义决策变量
    r_U = model_RT.addVars(nodes,  lb=0, name='r_U')       # 实际上调调整
    r_D = model_RT.addVars(nodes,  lb=0, name='r_D')       # 实际下调调整
    P_lsh = model_RT.addVars(nodes,  lb=0, name='P_lsh')   # 负荷削减
    delta_RT = model_RT.addVars(nodes,  lb=-180, ub=180, name='delta_RT') # 实时相角
    s = model_RT.addVars(nodes,  lb=0, name='s')           # 松弛变量
    
    # 目标函数
    obj_RT = gp.quicksum(
        (r_U[i]-r_D[i])*unit.iloc[i,2] 
        for i in range(nodes) 
    )
    obj_RT += gp.quicksum(P_lsh[i]*c for i in range(nodes) )
    obj_RT += gp.quicksum(s[i]*c_s for i in range(nodes) )
    model_RT.setObjective(obj_RT, GRB.MINIMIZE)
    
    # 约束条件
        # 实时市场功率平衡
    for t in range(nodes):
        model_RT.addConstr(
            r_U[t] - r_D[t] - s[t] - Y_real[t] + P_l_predict[t] + P_lsh[t] - 
            gp.quicksum(
                topo.iloc[t,j]*(delta_RT[t]-delta_RT[j]) - 
                topo.iloc[t,j]*(delta_DA[t].X-delta_DA[j].X) 
                for j in range(nodes)
            ) == 0
        )
        # 实时备用调整约束
    for i in range(nodes):
        model_RT.addConstr(r_U[i] <= R_U[i].X)  # 实际上调调整不超过备用
        model_RT.addConstr(r_D[i] <= R_D[i].X)  # 实际下调调整不超过备用
    model_RT.addConstr(delta_RT[0] == 0)  # 参考节点相角
        # 实时线路容量约束
    for i in range(nodes):
        for j in range(nodes):
            if i != j and topo.iloc[i,j] < -0.01:
                model_RT.addConstr(topo.iloc[i,j] * (delta_RT[i] - delta_RT[j]) <= 175)
                model_RT.addConstr(topo.iloc[i,j] * (delta_RT[i] - delta_RT[j]) >= -175)
    
    model_RT.optimize()
    
    # 计算总成本
    total_cost = model_DA.ObjVal + model_RT.ObjVal
    
    # 打印两阶段成本值和总成本值
    print(f"日前调度成本: {model_DA.ObjVal}")
    print(f"实时调度成本: {model_RT.ObjVal}")
    print(f"总成本: {total_cost}")
    
    # 释放模型资源
    model_DA.dispose()
    model_RT.dispose()
    
    return total_cost

# System parameters (topology data and generator unit data)
topo = pd.read_excel('118nodes_system.xlsx', sheet_name='topology', index_col=None, header=None) 
unit = pd.read_excel('118nodes_system.xlsx', sheet_name='unit')

# 遍历 hour 文件夹中的 1-24 文件夹
hour_folder = './hour'
for hour in range(22, 25):
    hour_dir = os.path.join(hour_folder, str(hour))
    if not os.path.exists(hour_dir):
        continue
    
    # 读取 allocated_output_perfect 文件
    perfect_file = os.path.join(hour_dir, 'allocated_output_perfect.csv')
    if not os.path.exists(perfect_file):
        continue
    
    # 读取 allocated_output_perfect 文件的内容
    with open(perfect_file, 'r') as f_perfect:
        reader_perfect = csv.reader(f_perfect)
        next(reader_perfect)  # 跳过标题行
        perfect_data = [list(map(float, row)) for row in reader_perfect]
    
    # 顺序计算无偏预测的总成本
    perfect_costs = []
    for data in perfect_data:
        perfect_costs.append(calculate_optimization_cost_two_stage(data, data, topo, unit))
    
    # 保存无偏预测的总成本到 CSV 文件
    perfect_cost_file = os.path.join(hour_dir, 'perfect_costs.csv')
    with open(perfect_cost_file, 'w', newline='') as f_perfect_cost:
        writer = csv.writer(f_perfect_cost)
        writer.writerow(['Perfect_Cost'])
        writer.writerows([[cost] for cost in perfect_costs])
    print(f"无偏预测的总成本已保存到 {perfect_cost_file} 文件中。")
    
    # 读取 allocated_output_change1-5 的 CSV 文件
    cost_differences = []  # 存储五个文件的成本差值
    for file_prefix in ['allocated_output_change1']:
        input_file = os.path.join(hour_dir, f'{file_prefix}.csv')
        if not os.path.exists(input_file):
            continue
        
        # 读取 CSV 文件
        with open(input_file, 'r') as f_in:
            reader = csv.reader(f_in)
            next(reader)  # 跳过标题行
            data = [list(map(float, row)) for row in reader]
        
        # 检查数据行数是否一致
        if len(data) != len(perfect_data):
            print(f"警告：{input_file} 文件的行数与 allocated_output_perfect.csv 不一致，跳过该文件。")
            continue
        
        # 顺序计算总成本
        total_costs = []
        for i in range(len(data)):
            total_costs.append(calculate_optimization_cost_two_stage(data[i], perfect_data[i], topo, unit))
        
        # 计算成本差值
        cost_diff = [total_costs[i] - perfect_costs[i] for i in range(len(total_costs))]
        cost_differences.append(cost_diff)
    
    # 将五个文件的成本差值合并为一个 CSV 文件
    if cost_differences:
        output_file = os.path.join(hour_dir, 'cost_differences.csv')
        with open(output_file, 'w', newline='') as f_out:
            writer = csv.writer(f_out)
            writer.writerow(['Change1'])
            writer.writerows(zip(*cost_differences))
        print(f"成本差值已保存到 {output_file} 文件中。")
    
    # 输出第 hour 文件夹已处理完成
    print(f"第 {hour} 文件夹已处理完成。")

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 10 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 1309 rows, 472 columns and 2971 nonzeros
Model fingerprint: 0x32fdfb27
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [4e+01, 2e+02]
  Bounds range     [2e+02, 2e+02]
  RHS range        [7e-01, 8e+02]
Presolve removed 1059 rows and 293 columns
Presolve time: 0.01s
Presolved: 250 rows, 301 columns, 1131 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -7.6180346e+08   2.662092e+06   0.000000e+00      0s
     137    8.4989477e+04   0.000000e+00   0.000000e+00      0s

Solved in 137 iterations and 0.01 seconds
Optimal objective  8.498947688e+04
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 10 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 1071 rows, 590 columns and 2617 nonzeros
Model fingerprint: 0x5323aa23
Coe

In [3]:
import os
import pandas as pd

# 定义输入和输出文件夹
input_folder = './generated_data'  # 当前目录
output_folder = 'load_variation'  # 输出文件夹

# 如果输出文件夹不存在，则创建
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 处理每个 output 文件
for i in range(1, 25):
    input_file = os.path.join(input_folder, f'output_{i}.csv')
    output_file = os.path.join(output_folder, f'load_variation_{i}.csv')
    
    # 读取 CSV 文件
    if os.path.exists(input_file):
        df = pd.read_csv(input_file)
        
        # 检查是否包含所需的列
        required_columns = ['load (MW)', 'Change1']
        if all(col in df.columns for col in required_columns):
            # 计算 load variation
            load_variation = df[['Change1']].sub(df['load (MW)'], axis=0)
            
            # 保存到新的 CSV 文件
            load_variation.to_csv(output_file, index=False)
            print(f"文件 {input_file} 处理完成，结果已保存到 {output_file}")
        else:
            print(f"文件 {input_file} 缺少必要的列，跳过处理。")
    else:
        print(f"文件 {input_file} 不存在，跳过处理。")

print("所有文件处理完成。")

文件 ./generated_data\output_1.csv 处理完成，结果已保存到 load_variation\load_variation_1.csv
文件 ./generated_data\output_2.csv 处理完成，结果已保存到 load_variation\load_variation_2.csv
文件 ./generated_data\output_3.csv 处理完成，结果已保存到 load_variation\load_variation_3.csv
文件 ./generated_data\output_4.csv 处理完成，结果已保存到 load_variation\load_variation_4.csv
文件 ./generated_data\output_5.csv 处理完成，结果已保存到 load_variation\load_variation_5.csv
文件 ./generated_data\output_6.csv 处理完成，结果已保存到 load_variation\load_variation_6.csv
文件 ./generated_data\output_7.csv 处理完成，结果已保存到 load_variation\load_variation_7.csv
文件 ./generated_data\output_8.csv 处理完成，结果已保存到 load_variation\load_variation_8.csv
文件 ./generated_data\output_9.csv 处理完成，结果已保存到 load_variation\load_variation_9.csv
文件 ./generated_data\output_10.csv 处理完成，结果已保存到 load_variation\load_variation_10.csv
文件 ./generated_data\output_11.csv 处理完成，结果已保存到 load_variation\load_variation_11.csv
文件 ./generated_data\output_12.csv 处理完成，结果已保存到 load_variation\load_variation_12.csv
文件 ./generated_data\ou

图像已保存到 ./plots\hour_1_fit_plot.png
图像已保存到 ./plots\hour_2_fit_plot.png
图像已保存到 ./plots\hour_3_fit_plot.png
图像已保存到 ./plots\hour_4_fit_plot.png
图像已保存到 ./plots\hour_5_fit_plot.png
图像已保存到 ./plots\hour_6_fit_plot.png
图像已保存到 ./plots\hour_7_fit_plot.png
图像已保存到 ./plots\hour_8_fit_plot.png
图像已保存到 ./plots\hour_9_fit_plot.png
图像已保存到 ./plots\hour_10_fit_plot.png
图像已保存到 ./plots\hour_11_fit_plot.png
图像已保存到 ./plots\hour_12_fit_plot.png
图像已保存到 ./plots\hour_13_fit_plot.png
图像已保存到 ./plots\hour_14_fit_plot.png
图像已保存到 ./plots\hour_15_fit_plot.png
图像已保存到 ./plots\hour_16_fit_plot.png
图像已保存到 ./plots\hour_17_fit_plot.png
图像已保存到 ./plots\hour_18_fit_plot.png
图像已保存到 ./plots\hour_19_fit_plot.png
图像已保存到 ./plots\hour_20_fit_plot.png
图像已保存到 ./plots\hour_21_fit_plot.png
图像已保存到 ./plots\hour_22_fit_plot.png
图像已保存到 ./plots\hour_23_fit_plot.png
图像已保存到 ./plots\hour_24_fit_plot.png
结果已保存到 ./linear_fit_results.csv 文件中。


In [11]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# 定义函数将多列数据连接为一列
def concatenate_columns(df):
    return df.values.flatten()

# 定义函数进行线性拟合
def fit_linear_regression(x, y):
    model = LinearRegression()
    model.fit(x.reshape(-1, 1), y)
    return model.coef_[0], model.intercept_

# 定义函数在约束条件下拟合低段或高段
def fit_constrained_segment(x, y, split_point, y_constraint):
    # 将截距表示为斜率的函数：intercept = y_constraint - slope * split_point
    # 目标是最小化数据点到拟合直线的误差
    # 定义目标函数：误差 = sum((y_i - (slope * x_i + intercept))^2)
    # 由于 intercept = y_constraint - slope * split_point，代入后：
    # 误差 = sum((y_i - (slope * x_i + y_constraint - slope * split_point))^2)
    # 展开并最小化斜率 slope
    x_centered = x - split_point
    slope = np.sum(x_centered * (y - y_constraint)) / np.sum(x_centered**2)
    intercept = y_constraint - slope * split_point
    return slope, intercept

# 定义主函数处理每个文件夹和对应的load_variation文件
def process_data(load_variation_dir, hour_dir, output_file):
    results = []
    plots_dir = './plots'  # 保存图像的目录
    os.makedirs(plots_dir, exist_ok=True)  # 创建目录
    # 固定分段点
    negative_split_point = -150
    positive_split_point = 150
    # 遍历1到24
    for i in range(1, 25):
        # 加载load_variation文件
        load_variation_file = os.path.join(load_variation_dir, f'load_variation_{i}.csv')
        if not os.path.exists(load_variation_file):
            print(f"文件 {load_variation_file} 不存在，跳过。")
            continue
        load_variation_data = pd.read_csv(load_variation_file)
        load_variation_concatenated = concatenate_columns(load_variation_data)
        # 加载cost_differences文件
        cost_differences_file = os.path.join(hour_dir, str(i), 'cost_differences.csv')
        if not os.path.exists(cost_differences_file):
            print(f"文件 {cost_differences_file} 不存在，跳过。")
            continue
        cost_differences_data = pd.read_csv(cost_differences_file)
        cost_differences_concatenated = concatenate_columns(cost_differences_data)
        # 将数据转换为numpy数组
        x = np.array(load_variation_concatenated)
        y = np.array(cost_differences_concatenated)
        # 按负荷偏差量为负和正分别进行分段拟合
        # 负侧分为两段
        x_negative = x[x < 0]
        y_negative = y[x < 0]
        if len(x_negative) > 0:
            # 按-150分段
            x_negative_low = x_negative[x_negative <= negative_split_point]
            y_negative_low = y_negative[x_negative <= negative_split_point]
            x_negative_high = x_negative[x_negative > negative_split_point]
            y_negative_high = y_negative[x_negative > negative_split_point]
            # 首先拟合高段
            if len(x_negative_high) > 0:
                slope_negative_high, intercept_negative_high = fit_linear_regression(x_negative_high, y_negative_high)
                # 计算高段在 x = -150 处的值
                y_neg_split_point_high = slope_negative_high * negative_split_point + intercept_negative_high
                # 在约束条件下拟合低段
                if len(x_negative_low) > 0:
                    slope_negative_low, intercept_negative_low = fit_constrained_segment(
                        x_negative_low, y_negative_low, negative_split_point, y_neg_split_point_high
                    )
                else:
                    slope_negative_low, intercept_negative_low = np.nan, np.nan
            else:
                slope_negative_high, intercept_negative_high = np.nan, np.nan
                slope_negative_low, intercept_negative_low = np.nan, np.nan
        else:
            slope_negative_low, intercept_negative_low = np.nan, np.nan
            slope_negative_high, intercept_negative_high = np.nan, np.nan
        # 正侧分为两段
        x_positive = x[x >= 0]
        y_positive = y[x >= 0]
        if len(x_positive) > 0:
            # 按+150分段
            x_positive_low = x_positive[x_positive <= positive_split_point]
            y_positive_low = y_positive[x_positive <= positive_split_point]
            x_positive_high = x_positive[x_positive > positive_split_point]
            y_positive_high = y_positive[x_positive > positive_split_point]
            # 首先拟合低段
            if len(x_positive_low) > 0:
                slope_positive_low, intercept_positive_low = fit_linear_regression(x_positive_low, y_positive_low)
                # 计算低段在 x = 150 处的值
                y_pos_split_point_low = slope_positive_low * positive_split_point + intercept_positive_low
                # 在约束条件下拟合高段
                if len(x_positive_high) > 0:
                    slope_positive_high, intercept_positive_high = fit_constrained_segment(
                        x_positive_high, y_positive_high, positive_split_point, y_pos_split_point_low
                    )
                else:
                    slope_positive_high, intercept_positive_high = np.nan, np.nan
            else:
                slope_positive_low, intercept_positive_low = np.nan, np.nan
                slope_positive_high, intercept_positive_high = np.nan, np.nan
        else:
            slope_positive_low, intercept_positive_low = np.nan, np.nan
            slope_positive_high, intercept_positive_high = np.nan, np.nan
        # 将结果存入列表
        results.append({
            'Hour': i,
            'Negative_Slope_Low': slope_negative_low,
            'Negative_Intercept_Low': intercept_negative_low,
            'Negative_Slope_High': slope_negative_high,
            'Negative_Intercept_High': intercept_negative_high,
            'Positive_Slope_Low': slope_positive_low,
            'Positive_Intercept_Low': intercept_positive_low,
            'Positive_Slope_High': slope_positive_high,
            'Positive_Intercept_High': intercept_positive_high
        })
        # 绘图
        plt.figure(figsize=(10, 6))
        plt.scatter(x, y, color='blue', label='Data Points')
        # 负侧拟合线
        if not np.isnan(slope_negative_low):
            x_fit_negative_low = np.linspace(min(x_negative), negative_split_point, 100)
            y_fit_negative_low = slope_negative_low * x_fit_negative_low + intercept_negative_low
            plt.plot(x_fit_negative_low, y_fit_negative_low, color='red', label='Negative Fit Low')
        if not np.isnan(slope_negative_high):
            x_fit_negative_high = np.linspace(negative_split_point, 0, 100)
            y_fit_negative_high = slope_negative_high * x_fit_negative_high + intercept_negative_high
            plt.plot(x_fit_negative_high, y_fit_negative_high, color='orange', label='Negative Fit High')
        # 正侧拟合线
        if not np.isnan(slope_positive_low):
            x_fit_positive_low = np.linspace(0, positive_split_point, 100)
            y_fit_positive_low = slope_positive_low * x_fit_positive_low + intercept_positive_low
            plt.plot(x_fit_positive_low, y_fit_positive_low, color='green', label='Positive Fit Low')
        if not np.isnan(slope_positive_high):
            x_fit_positive_high = np.linspace(positive_split_point, max(x_positive), 100)
            y_fit_positive_high = slope_positive_high * x_fit_positive_high + intercept_positive_high
            plt.plot(x_fit_positive_high, y_fit_positive_high, color='purple', label='Positive Fit High')
        plt.title(f'Hour {i} - Load Deviation vs Cost Deviation (Segmented Fit at ±150)')
        plt.xlabel('Load Deviation')
        plt.ylabel('Cost Deviation')
        plt.legend()
        plt.grid(True)
        # 保存图像
        plot_file = os.path.join(plots_dir, f'hour_{i}_segmented_fit_plot.png')
        plt.savefig(plot_file)
        plt.close()
        print(f"图像已保存到 {plot_file}")
    # 将结果保存为CSV文件
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_file, index=False)
    print(f"结果已保存到 {output_file} 文件中。")

# 定义路径
load_variation_dir = './load_variation'  # load_variation文件所在的目录
hour_dir = './hour'  # hour文件夹所在的目录
output_file = './segmented_linear_fit_results.csv'  # 输出文件路径

# 执行主函数
process_data(load_variation_dir, hour_dir, output_file)

图像已保存到 ./plots\hour_1_segmented_fit_plot.png
图像已保存到 ./plots\hour_2_segmented_fit_plot.png
图像已保存到 ./plots\hour_3_segmented_fit_plot.png
图像已保存到 ./plots\hour_4_segmented_fit_plot.png
图像已保存到 ./plots\hour_5_segmented_fit_plot.png
图像已保存到 ./plots\hour_6_segmented_fit_plot.png
图像已保存到 ./plots\hour_7_segmented_fit_plot.png
图像已保存到 ./plots\hour_8_segmented_fit_plot.png
图像已保存到 ./plots\hour_9_segmented_fit_plot.png
图像已保存到 ./plots\hour_10_segmented_fit_plot.png
图像已保存到 ./plots\hour_11_segmented_fit_plot.png
图像已保存到 ./plots\hour_12_segmented_fit_plot.png
图像已保存到 ./plots\hour_13_segmented_fit_plot.png
图像已保存到 ./plots\hour_14_segmented_fit_plot.png
图像已保存到 ./plots\hour_15_segmented_fit_plot.png
图像已保存到 ./plots\hour_16_segmented_fit_plot.png
图像已保存到 ./plots\hour_17_segmented_fit_plot.png
图像已保存到 ./plots\hour_18_segmented_fit_plot.png
图像已保存到 ./plots\hour_19_segmented_fit_plot.png
图像已保存到 ./plots\hour_20_segmented_fit_plot.png
图像已保存到 ./plots\hour_21_segmented_fit_plot.png
图像已保存到 ./plots\hour_22_segmented_fit_plot.p

In [4]:
import os
import pandas as pd

# 定义函数将数据按小时分组并保存为24个CSV文件
def split_and_save_data(X_train_file, Y_train_file, output_dir):
    # 加载X_train和Y_train数据
    X_train = pd.read_csv(X_train_file)
    Y_train = pd.read_csv(Y_train_file)

    # 确保X_train和Y_train的行数相同
    if len(X_train) != len(Y_train):
        raise ValueError("X_train和Y_train的行数不一致，请检查数据。")

    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)

    # 按小时分组
    for i in range(24):
        # 提取符合条件的行
        extracted_X = X_train[(X_train.index - i) % 24 == 0]
        extracted_Y = Y_train[(Y_train.index - i) % 24 == 0]

        # 确保提取的行数一致
        if len(extracted_X) != len(extracted_Y):
            raise ValueError(f"提取的第{i}小时数据行数不一致，请检查数据。")

        # 保存为CSV文件
        X_output_file = os.path.join(output_dir, f'X_train_hour_{i + 1}.csv')
        Y_output_file = os.path.join(output_dir, f'Y_train_hour_{i + 1}.csv')
        extracted_X.to_csv(X_output_file, index=False)
        extracted_Y.to_csv(Y_output_file, index=False)
        print(f"第{i + 1}小时的数据已保存到 {X_output_file} 和 {Y_output_file} 文件中。")

# 定义文件路径
X_train_file = './X_train.csv'  # X_train文件路径
Y_train_file = './Y_train.csv'  # Y_train文件路径
output_dir = './hourly_data'    # 保存分组数据的目录

# 执行函数
split_and_save_data(X_train_file, Y_train_file, output_dir)

第1小时的数据已保存到 ./hourly_data\X_train_hour_1.csv 和 ./hourly_data\Y_train_hour_1.csv 文件中。
第2小时的数据已保存到 ./hourly_data\X_train_hour_2.csv 和 ./hourly_data\Y_train_hour_2.csv 文件中。
第3小时的数据已保存到 ./hourly_data\X_train_hour_3.csv 和 ./hourly_data\Y_train_hour_3.csv 文件中。
第4小时的数据已保存到 ./hourly_data\X_train_hour_4.csv 和 ./hourly_data\Y_train_hour_4.csv 文件中。
第5小时的数据已保存到 ./hourly_data\X_train_hour_5.csv 和 ./hourly_data\Y_train_hour_5.csv 文件中。
第6小时的数据已保存到 ./hourly_data\X_train_hour_6.csv 和 ./hourly_data\Y_train_hour_6.csv 文件中。
第7小时的数据已保存到 ./hourly_data\X_train_hour_7.csv 和 ./hourly_data\Y_train_hour_7.csv 文件中。
第8小时的数据已保存到 ./hourly_data\X_train_hour_8.csv 和 ./hourly_data\Y_train_hour_8.csv 文件中。
第9小时的数据已保存到 ./hourly_data\X_train_hour_9.csv 和 ./hourly_data\Y_train_hour_9.csv 文件中。
第10小时的数据已保存到 ./hourly_data\X_train_hour_10.csv 和 ./hourly_data\Y_train_hour_10.csv 文件中。
第11小时的数据已保存到 ./hourly_data\X_train_hour_11.csv 和 ./hourly_data\Y_train_hour_11.csv 文件中。
第12小时的数据已保存到 ./hourly_data\X_train_hour_12.csv 和 ./hourly_d

In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp

# 读取分段斜率和截距的文件
fit_results_df = pd.read_csv('./segmented_linear_fit_results.csv')

# 定义函数训练单个小时的线性回归模型
def train_hourly_model(hour, X_train, y_train, a_neg_low, b_neg_low, a_neg_high, b_neg_high, a_pos_low, b_pos_low, a_pos_high, b_pos_high):
    # 创建 Gurobi 模型
    model = gp.Model(f'hour_{hour}_linear_regression')
    model.setParam('OutputFlag', 0)  # 关闭输出日志

    # 定义决策变量 theta 和 beta
    theta = model.addVars(X_train.shape[1], lb=-1000, vtype=gp.GRB.CONTINUOUS, name='theta')
    beta = model.addVar(lb=-10000, vtype=gp.GRB.CONTINUOUS, name='beta')

    # 定义中间变量 t
    t = model.addVars(len(y_train), vtype=gp.GRB.CONTINUOUS, name='t')

    # 设置目标函数：最小化 t 的均值
    model.setObjective(gp.quicksum(t[s] for s in range(len(y_train))) / len(y_train), gp.GRB.MINIMIZE)

    # 添加四个约束条件
    for p in range(len(y_train)):
        # 负侧低段约束
        model.addConstr(
            a_neg_low * (gp.quicksum(theta[i] * X_train.iloc[p, i] for i in range(X_train.shape[1])) + beta - y_train.iloc[p]) + b_neg_low <= t[p],
            name=f'constraint_neg_low_{p}'
        )
        # 负侧高段约束
        model.addConstr(
            a_neg_high * (gp.quicksum(theta[i] * X_train.iloc[p, i] for i in range(X_train.shape[1])) + beta - y_train.iloc[p]) + b_neg_high <= t[p],
            name=f'constraint_neg_high_{p}'
        )
        # 正侧低段约束
        model.addConstr(
            a_pos_low * (gp.quicksum(theta[i] * X_train.iloc[p, i] for i in range(X_train.shape[1])) + beta - y_train.iloc[p]) + b_pos_low <= t[p],
            name=f'constraint_pos_low_{p}'
        )
        # 正侧高段约束
        model.addConstr(
            a_pos_high * (gp.quicksum(theta[i] * X_train.iloc[p, i] for i in range(X_train.shape[1])) + beta - y_train.iloc[p]) + b_pos_high <= t[p],
            name=f'constraint_pos_high_{p}'
        )

    # 优化模型
    model.optimize()

    # 获取优化后的 theta 和 beta 值
    theta_result = np.array([theta[i].x for i in range(X_train.shape[1])])
    beta_result = beta.x

    # 获取优化后的 t 值
    t_values = np.array([t[p].x for p in range(len(y_train))])

    return theta_result, beta_result, t_values

# 主函数
def train_all_hourly_models():
    hourly_theta_results = {}
    hourly_beta_results = {}
    hourly_t_values = {}

    for hour in range(1, 25):
        # 加载对应小时的训练数据
        X_train_file = f'./hourly_data/X_train_hour_{hour}.csv'
        Y_train_file = f'./hourly_data/Y_train_hour_{hour}.csv'
        X_train = pd.read_csv(X_train_file)
        y_train = pd.read_csv(Y_train_file)

        # 获取对应小时的四组斜率和截距
        fit_results = fit_results_df[fit_results_df['Hour'] == hour].iloc[0]
        a_neg_low = fit_results['Negative_Slope_Low']
        b_neg_low = fit_results['Negative_Intercept_Low']
        a_neg_high = fit_results['Negative_Slope_High']
        b_neg_high = fit_results['Negative_Intercept_High']
        a_pos_low = fit_results['Positive_Slope_Low']
        b_pos_low = fit_results['Positive_Intercept_Low']
        a_pos_high = fit_results['Positive_Slope_High']
        b_pos_high = fit_results['Positive_Intercept_High']

        # 训练对应小时的模型
        theta_result, beta_result, t_values = train_hourly_model(hour, X_train, y_train, a_neg_low, b_neg_low, a_neg_high, b_neg_high, a_pos_low, b_pos_low, a_pos_high, b_pos_high)

        # 保存结果
        hourly_theta_results[hour] = theta_result
        hourly_beta_results[hour] = beta_result
        hourly_t_values[hour] = t_values

        # 打印结果
        print(f"第{hour}小时优化后的 theta 值：", theta_result)
        print(f"第{hour}小时优化后的 beta 值：", beta_result)

    # 将 theta 结果保存到 DataFrame
    theta_results_df = pd.DataFrame.from_dict(hourly_theta_results, orient='index')
    theta_results_df.to_csv('hourly_theta_results.csv', index_label='Hour')
    print("各小时的 theta 值已保存到 hourly_theta_results.csv 文件中。")

    # 将 beta 结果保存到 DataFrame
    beta_results_df = pd.DataFrame.from_dict(hourly_beta_results, orient='index', columns=['Beta'])
    beta_results_df.to_csv('hourly_beta_results.csv', index_label='Hour')
    print("各小时的 beta 值已保存到 hourly_beta_results.csv 文件中。")

    # 将 t 值保存到 DataFrame
    t_values_df = pd.DataFrame.from_dict(hourly_t_values, orient='index')
    t_values_df.to_csv('hourly_t_values.csv', index_label='Hour')
    print("各小时的 t 值已保存到 hourly_t_values.csv 文件中。")

# 执行主函数
train_all_hourly_models()

# 加载训练好的 theta 和 beta 值
theta_results_df = pd.read_csv('hourly_theta_results.csv', index_col='Hour')
beta_results_df = pd.read_csv('hourly_beta_results.csv', index_col='Hour')

# 定义函数预测单个小时的 Y_train 值
def predict_hourly_y(hour, X_train, theta, beta):
    # 使用 theta 和 beta 进行线性预测
    y_pred = np.dot(X_train, theta) + beta
    return y_pred

# 主函数
def combine_and_save_predictions():
    # 初始化一个空列表，用于存储所有预测值
    all_y_pred = []

    for hour in range(1, 25):
        # 加载对应小时的训练数据
        X_train_file = f'./hourly_data/X_train_hour_{hour}.csv'
        X_train = pd.read_csv(X_train_file)

        # 获取对应小时的 theta 和 beta 值
        theta = theta_results_df.loc[hour].values
        beta = beta_results_df.loc[hour, 'Beta']

        # 预测对应小时的 Y_train 值
        y_pred = predict_hourly_y(hour, X_train, theta, beta)

        # 将预测值添加到列表中
        all_y_pred.append(y_pred)

    # 将预测值按原始顺序重新组合
    combined_y_pred = np.zeros(len(all_y_pred) * len(all_y_pred[0]))
    for hour in range(24):
        for j in range(len(all_y_pred[hour])):
            combined_y_pred[j * 24 + hour] = all_y_pred[hour][j]

    # 将组合后的预测值保存为 CSV 文件
    combined_y_pred_df = pd.DataFrame(combined_y_pred, columns=['Predicted_Y'])
    combined_y_pred_df.to_csv('combined_y_train_predictions.csv', index=False)
    print("组合后的 Y_train 预测值已保存到 combined_y_train_predictions.csv 文件中。")

# 执行主函数
combine_and_save_predictions()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-30
第1小时优化后的 theta 值： [   0.66226782  -16.20967462 -594.89417359    2.93746532   -6.5782059
   18.6331991    12.57707651  -12.32390869  -41.36660072  -19.11684186
   -2.56842665   15.79281419   12.19180378   -7.84790849]
第1小时优化后的 beta 值： 3415.2172846922276


C:\Users\85033\AppData\Local\Temp\ipykernel_34840\3915162986.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_neg_low * (gp.quicksum(theta[i] * X_train.iloc[p, i] for i in range(X_train.shape[1])) + beta - y_train.iloc[p]) + b_neg_low <= t[p],
C:\Users\85033\AppData\Local\Temp\ipykernel_34840\3915162986.py:33: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_neg_high * (gp.quicksum(theta[i] * X_train.iloc[p, i] for i in range(X_train.shape[1])) + beta - y_train.iloc[p]) + b_neg_high <= t[p],
C:\Users\85033\AppData\Local\Temp\ipykernel_34840\3915162986.py:38: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_pos_low * (gp.quicksum(theta[i] * X_train.iloc[p, i] for i in range(X_train.shape[1])

第2小时优化后的 theta 值： [-1.91542889e+00  2.42174708e+01  1.32906698e+03  6.15729135e+00
  2.37819736e+00  9.92466801e+00  9.14355324e+00 -6.43815163e+00
 -3.36260442e+01 -1.06363899e+01  1.14565237e+00  9.07224246e+00
 -6.63573807e+00 -9.97989689e+00]
第2小时优化后的 beta 值： 1419.9345879279153
第3小时优化后的 theta 值： [ -1.42363137  11.74474195 987.39604778   4.37969928  25.61441309
  22.69663395   4.46591131 -22.63413603 -42.28239424 -14.83417655
   5.23030793  10.78529432  -4.04960673 -23.94658527]
第3小时优化后的 beta 值： 2012.043747024225


C:\Users\85033\AppData\Local\Temp\ipykernel_34840\3915162986.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_neg_low * (gp.quicksum(theta[i] * X_train.iloc[p, i] for i in range(X_train.shape[1])) + beta - y_train.iloc[p]) + b_neg_low <= t[p],
C:\Users\85033\AppData\Local\Temp\ipykernel_34840\3915162986.py:33: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_neg_high * (gp.quicksum(theta[i] * X_train.iloc[p, i] for i in range(X_train.shape[1])) + beta - y_train.iloc[p]) + b_neg_high <= t[p],
C:\Users\85033\AppData\Local\Temp\ipykernel_34840\3915162986.py:38: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_pos_low * (gp.quicksum(theta[i] * X_train.iloc[p, i] for i in range(X_train.shape[1])

第4小时优化后的 theta 值： [ -1.11076266   6.43726773 748.48992066   2.58742585  22.65910824
   8.17671367  17.78701782 -13.59345527 -57.96765925 -29.51080869
  -3.23747878  27.33949081  17.44036798 -25.66583321]
第4小时优化后的 beta 值： 2251.592823136936
第5小时优化后的 theta 值： [ -0.9108539    4.21299639 595.01594372  14.96767272  11.95708603
   1.58558827   9.08959835 -11.63095133 -44.79114809 -35.14023127
   4.48093045  39.76984551  -4.14016541 -21.24356864]
第5小时优化后的 beta 值： 2481.718862373751


...

第20小时优化后的 theta 值： [-5.22316989e+00  6.23539488e+00  3.83215732e+03  1.35526468e+01
 -5.30540166e+00  7.33371543e+00  2.04536904e+00 -1.37356880e+01
 -4.93172191e+01 -1.48489417e+00  2.87323655e+00 -5.07596847e+00
  2.04050475e+01 -3.50282640e+00]
第20小时优化后的 beta 值： -520.274882719068
第21小时优化后的 theta 值： [-4.64509545e+00  5.16708596e+00  3.32074823e+03 -2.79696975e+00
  2.17998036e+01  1.70222381e+01 -3.15689325e+00 -3.35013809e+00
 -1.87064883e+01 -2.10691639e+00  1.08527143e+01  1.40904893e+01
 -2.71057451e+01 -3.32006224e+01]
第21小时优化后的 beta 值： -77.36070948234556


C:\Users\85033\AppData\Local\Temp\ipykernel_34840\3915162986.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_neg_low * (gp.quicksum(theta[i] * X_train.iloc[p, i] for i in range(X_train.shape[1])) + beta - y_train.iloc[p]) + b_neg_low <= t[p],
C:\Users\85033\AppData\Local\Temp\ipykernel_34840\3915162986.py:33: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_neg_high * (gp.quicksum(theta[i] * X_train.iloc[p, i] for i in range(X_train.shape[1])) + beta - y_train.iloc[p]) + b_neg_high <= t[p],
C:\Users\85033\AppData\Local\Temp\ipykernel_34840\3915162986.py:38: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_pos_low * (gp.quicksum(theta[i] * X_train.iloc[p, i] for i in range(X_train.shape[1])

第22小时优化后的 theta 值： [-3.02602189e+00  3.28478461e+00  2.17611439e+03  7.15210974e+00
 -1.07883291e+01  3.16447465e+01  2.60794884e+00 -2.76492473e+01
 -2.89273220e+01 -2.03024071e+01  1.97114211e-01  1.54237468e+01
  6.40341267e+00 -2.03468443e+00]
第22小时优化后的 beta 值： 825.2125448515508
第23小时优化后的 theta 值： [ -1.2017942    1.22519379 903.53331887   8.91229855  -2.16311702
  11.01585444   9.19796839 -13.90671231 -51.92575737 -15.30323464
  -5.49893513  14.1994422   25.33607953  -9.5235947 ]
第23小时优化后的 beta 值： 2048.7293508407092


C:\Users\85033\AppData\Local\Temp\ipykernel_34840\3915162986.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_neg_low * (gp.quicksum(theta[i] * X_train.iloc[p, i] for i in range(X_train.shape[1])) + beta - y_train.iloc[p]) + b_neg_low <= t[p],
C:\Users\85033\AppData\Local\Temp\ipykernel_34840\3915162986.py:33: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_neg_high * (gp.quicksum(theta[i] * X_train.iloc[p, i] for i in range(X_train.shape[1])) + beta - y_train.iloc[p]) + b_neg_high <= t[p],
C:\Users\85033\AppData\Local\Temp\ipykernel_34840\3915162986.py:38: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_pos_low * (gp.quicksum(theta[i] * X_train.iloc[p, i] for i in range(X_train.shape[1])

第24小时优化后的 theta 值： [-2.07890821e+00  1.98356403e+00  1.47736942e+03  8.19911356e+00
 -6.32969416e+00 -1.21698382e+01  5.92482596e+00  6.08214492e+00
 -7.28669565e+01 -7.60333804e-01 -4.43584478e+00  1.10841323e+01
  3.45983067e+01 -2.24074605e+00]
第24小时优化后的 beta 值： 1553.9790167102965
各小时的 theta 值已保存到 hourly_theta_results.csv 文件中。
各小时的 beta 值已保存到 hourly_beta_results.csv 文件中。
各小时的 t 值已保存到 hourly_t_values.csv 文件中。
组合后的 Y_train 预测值已保存到 combined_y_train_predictions.csv 文件中。


In [3]:
import pandas as pd
import os

# 定义函数将 X_test 和 Y_test 按 24 小时分割并保存
def split_test_data_by_hour(X_test_file, Y_test_file, output_folder, start_hour=7):
    # 加载 X_test 和 Y_test 文件
    X_test = pd.read_csv(X_test_file)
    Y_test = pd.read_csv(Y_test_file)

    # 确保输出文件夹存在
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 按 24 小时分割数据
    for hour in range(24):
        # 计算实际的小时索引（从 start_hour 开始）
        actual_hour = (start_hour - 1 + hour) % 24 + 1

        # 提取符合条件的行
        extracted_X = X_test.iloc[hour::24, :]
        extracted_Y = Y_test.iloc[hour::24, :]

        # 保存提取的数据到对应的文件
        extracted_X.to_csv(f'{output_folder}/X_test_hour_{actual_hour}.csv', index=False)
        extracted_Y.to_csv(f'{output_folder}/Y_test_hour_{actual_hour}.csv', index=False)

    print(f"X_test 和 Y_test 已按 24 小时分割并保存到 {output_folder} 文件夹中。")

# 主函数
def main():
    # 定义 X_test 和 Y_test 文件路径
    X_test_file = 'X_test.csv'
    Y_test_file = 'Y_test.csv'

    # 定义输出文件夹
    output_folder = './test_split'

    # 调用函数分割数据，指定从 7 点开始
    split_test_data_by_hour(X_test_file, Y_test_file, output_folder, start_hour=7)

# 执行主函数
main()

X_test 和 Y_test 已按 24 小时分割并保存到 ./test_split 文件夹中。


In [6]:
import pandas as pd
import numpy as np
import os

# 定义函数进行单小时预测
def predict_hourly_y(hour, X_test, theta, beta):
    # 使用 theta 和 beta 进行线性预测
    y_pred = np.dot(X_test, theta) + beta
    return y_pred

# 定义函数将预测值按 trend 列排序并重新组合
def combine_and_sort_predictions(predictions, trend_values):
    # 将所有预测值合并为一个数组
    all_predictions = np.concatenate(predictions)
    # 将所有 trend 值合并为一个数组
    all_trends = np.concatenate(trend_values)
    # 根据 trend 值排序
    sorted_indices = np.argsort(all_trends)
    # 按排序后的顺序重新组合预测值
    sorted_predictions = all_predictions[sorted_indices]
    return sorted_predictions

# 主函数
def predict_and_combine():
    # 加载训练好的 theta 和 beta 值
    theta_results_df = pd.read_csv('hourly_theta_results.csv', index_col='Hour')
    beta_results_df = pd.read_csv('hourly_beta_results.csv', index_col='Hour')

    # 初始化一个列表，用于存储所有小时的预测值
    all_predictions = []
    # 初始化一个列表，用于存储所有小时的 trend 值
    all_trends = []

    # 遍历 24 小时
    for hour in range(1, 25):
        # 加载对应小时的测试数据
        X_test_file = f'./test_split/X_test_hour_{hour}.csv'
        X_test = pd.read_csv(X_test_file)

        # 获取对应小时的 theta 和 beta 值
        theta = theta_results_df.loc[hour].values
        beta = beta_results_df.loc[hour, 'Beta']

        # 进行预测
        y_pred = predict_hourly_y(hour, X_test, theta, beta)

        # 将预测值添加到列表中
        all_predictions.append(y_pred)
        # 将 trend 值添加到列表中
        all_trends.append(X_test['trend'].values)

    # 将预测值按 trend 列排序并重新组合
    sorted_predictions = combine_and_sort_predictions(all_predictions, all_trends)

    # 将排序后的预测值保存为 CSV 文件
    sorted_predictions_df = pd.DataFrame(sorted_predictions, columns=['Predicted_Y'])
    sorted_predictions_df.to_csv('sorted_y_test_predictions.csv', index=False)
    print("按 trend 列排序后的 Y_test 预测值已保存到 sorted_y_test_predictions.csv 文件中。")

# 执行主函数
predict_and_combine()

按 trend 列排序后的 Y_test 预测值已保存到 sorted_y_test_predictions.csv 文件中。
